<a href="https://colab.research.google.com/github/miguellucero123/Analisis_est/blob/main/Trabajo_S2_TAC_ML_Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ====== 1. LIBRERÍAS Y CONEXIÓN A DRIVE ======
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración visual global
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# ====== 2. CARGA DEL DATASET ======
ruta = '/content/drive/MyDrive/bank-additional-full.csv'  # Cambia la ruta si es necesario

try:
    df = pd.read_csv(ruta, sep=';')
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print(f"Error: archivo no encontrado en {ruta}")
    df = pd.DataFrame()
except Exception as e:
    print(f"Ocurrió un error: {e}")
    df = pd.DataFrame()

# Vista preliminar
display(df.head())
print(f"Dimensión del DataFrame: {df.shape}")

In [ ]:
# ====== 3. REVISIÓN DE CALIDAD DE DATOS ======
print("====== REVISIÓN DE CONSISTENCIA DE DATOS ======\n")

# 1. Valores nulos (NaN)
print("1. Valores nulos por columna:")
print(df.isnull().sum())
print("-" * 40)

# 2. Celdas en blanco ('' o sólo espacios)
print("2. Celdas en blanco por columna:")
for col in df.columns:
    blancos = df[col].apply(lambda x: isinstance(x, str) and x.strip() == '').sum()
    if blancos > 0:
        print(f"{col}: {blancos}")
print("-" * 40)

# 3. Valores negativos en columnas numéricas
print("3. Valores negativos detectados:")
numericas = df.select_dtypes(include=np.number).columns
for col in numericas:
    negativos = (df[col] < 0).sum()
    if negativos > 0:
        print(f"{col}: {negativos}")
print("-" * 40)

# 4. Problemas con fechas (si existen columnas tipo fecha)
if 'month' in df.columns:
    print("4. Valores únicos en 'month':", df['month'].unique())
if 'day' in df.columns:
    print("Valores únicos en 'day':", df['day'].unique())
print("-" * 40)

# 5. Valores inconsistentes en categóricas
cat_cols = ['marital', 'education', 'job', 'housing', 'loan', 'month', 'contact', 'y']
for col in cat_cols:
    if col in df.columns:
        print(f"5. Valores únicos en '{col}': {df[col].unique()}")
print("-" * 40)

# 6. Edades fuera de rango lógico
if 'age' in df.columns:
    fuera_rango_menor = df[df['age'] < 18]
    fuera_rango_mayor = df[df['age'] > 100]
    print(f"6. Registros con edad < 18: {len(fuera_rango_menor)}")
    if len(fuera_rango_menor) > 0:
        display(fuera_rango_menor[['age']])
    print(f"Registros con edad > 100: {len(fuera_rango_mayor)}")
    if len(fuera_rango_mayor) > 0:
        display(fuera_rango_mayor[['age']])
print("-" * 40)

# 7. Resumen rápido
print("====== RESUMEN DE INCONSISTENCIAS ======")
total_nan = df.isnull().sum().sum()
print(f"Total de valores NaN: {total_nan}")

total_blancos = sum(df[col].apply(lambda x: isinstance(x, str) and x.strip() == '').sum() for col in df.columns)
print(f"Total de celdas en blanco: {total_blancos}")

total_negativos = sum((df[col] < 0).sum() for col in numericas)
print(f"Total de valores negativos en columnas numéricas: {total_negativos}")

if 'age' in df.columns:
    print(f"Total de edades fuera de rango (<18): {len(fuera_rango_menor)}")
    print(f"Total de edades fuera de rango (>100): {len(fuera_rango_mayor)}")

print("\nSi encuentras inconsistencias, considera limpiar o corregir antes del análisis.")
print("=" * 50)

In [ ]:
# ====== 4. PREPROCESAMIENTO ======
if not df.empty:
    # a) Variable objetivo binaria
    if 'y' in df.columns:
        df['conversion'] = df['y'].map({'yes': 1, 'no': 0})
    else:
        print("No se encontró la columna 'y'.")

    # b) Grupos de edad
    bins = [18, 20, 30, 40, 50, 60, 70, np.inf]
    labels = ['<20','[20,30)','[30,40)','[40,50)','[50,60)','[60,70)','70+']
    df['grupo_edad'] = pd.cut(df['age'], bins=bins, labels=labels, right=False, include_lowest=True)

    # c) Imputar 'unknown' con moda en marital y education
    for col in ['marital', 'education']:
        if col in df.columns and 'unknown' in df[col].unique():
            moda = df[col].mode()[0]
            df[col] = df[col].replace('unknown', moda)

    # d) Tipos de datos correctos
    if 'age' in df.columns:
        df['age'] = df['age'].astype(int)
    if 'conversion' in df.columns:
        df['conversion'] = df['conversion'].astype(int)
    if 'grupo_edad' in df.columns:
        df['grupo_edad'] = df['grupo_edad'].astype('category')
    if 'marital' in df.columns:
        df['marital'] = df['marital'].astype('category')
    if 'education' in df.columns:
        df['education'] = df['education'].astype('category')

    print("Preprocesamiento completado.")
    display(df.head())
else:
    print("El DataFrame está vacío, no se puede preprocesar.")

In [ ]:
# ====== 5.1 CONVERSIÓN ABSOLUTA Y TASA DE CONVERSIÓN ======
if not df.empty and 'conversion' in df.columns:
    conversion_absoluta = df['conversion'].sum()
    tasa_conversion = df['conversion'].mean() * 100
    print(f"Conversión absoluta: {conversion_absoluta}")
    print(f"Tasa de conversión: {tasa_conversion:.2f}%")
else:
    print("No se puede calcular la conversión.")

In [ ]:
# ====== 5.2 TASA DE CONVERSIÓN POR EDAD Y GRUPO ETARIO ======
if not df.empty and 'conversion' in df.columns:
    print("\nTasa de conversión por edad específica:")
    print(df.groupby('age')['conversion'].mean() * 100)

    print("\nTasa de conversión por grupo etario:")
    print(df.groupby('grupo_edad', observed=False)['conversion'].mean() * 100)

In [ ]:
# ====== 5.3 TASA DE CONVERSIÓN POR ESTADO CIVIL ======
if not df.empty and 'conversion' in df.columns:
    print("\nTasa de conversión por estado marital:")
    print(df.groupby('marital', observed=False)['conversion'].mean() * 100)

In [ ]:
# ====== 5.4 TASA DE CONVERSIÓN POR ESTADO CIVIL Y GRUPO ETARIO ======
if not df.empty and 'conversion' in df.columns:
    print("\nTasa de conversión por estado marital y grupo etario:")
    tabla1 = df.groupby(['marital', 'grupo_edad'], observed=False)['conversion'].mean().unstack() * 100
    display(tabla1)

In [ ]:
# ====== 5.5 TASA DE CONVERSIÓN POR NIVEL EDUCATIVO ======
if not df.empty and 'conversion' in df.columns:
    print("\nTasa de conversión por nivel educativo:")
    print(df.groupby('education', observed=False)['conversion'].mean() * 100)

In [ ]:
# ====== 5.6 GRÁFICO TASA DE CONVERSIÓN POR NIVEL EDUCATIVO ======
if not df.empty and 'conversion' in df.columns:
    tasa_edu = df.groupby('education', observed=False)['conversion'].mean().sort_values() * 100
    tasa_edu.plot(kind='bar', color='steelblue')
    plt.title('Tasa de conversión por nivel educativo')
    plt.ylabel('Tasa de conversión (%)')
    plt.xlabel('Nivel educativo')
    plt.show()

In [ ]:
# ====== 5.7 TASA DE CONVERSIÓN POR NIVEL EDUCATIVO Y GRUPO ETARIO ======
if not df.empty and 'conversion' in df.columns:
    tabla2 = df.groupby(['education', 'grupo_edad'], observed=False)['conversion'].mean().unstack() * 100
    print("\nTasa de conversión por nivel educativo y grupo etario:")
    display(tabla2)

In [ ]:
# ====== 5.8 GRÁFICO CRUCE NIVEL EDUCATIVO Y GRUPO ETARIO ======
if not df.empty and 'conversion' in df.columns:
    plt.figure(figsize=(14,7))
    tabla2.T.plot(kind='bar')
    plt.title('Tasa de conversión por grupo etario y nivel educativo')
    plt.ylabel('Tasa de conversión (%)')
    plt.xlabel('Grupo etario')
    plt.legend(title='Nivel educativo', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
# Parte 6: Tasa conversión por nivel educativo
tasa_conversion_por_educacion = df.groupby('education')['conversion'].mean() * 100
print("\nTasa de conversión por educación:")
display(tasa_conversion_por_educacion)

In [ ]:
# Parte 7: Gráfico de barras para educación
import matplotlib.pyplot as plt

# Ordenar la tasa de conversión para visualización
tasa_conversion_por_educacion_ordenado = tasa_conversion_por_educacion.sort_values()

# Crear el gráfico de barras
plt.figure(figsize=(12, 6))  # Ajustar tamaño de figura para legibilidad
tasa_conversion_por_educacion_ordenado.plot(kind='bar', color='skyblue')
plt.title('Tasa de conversión por nivel educativo', fontsize=16)
plt.xlabel('Nivel educativo', fontsize=12)
plt.ylabel('Tasa de conversión (%)', fontsize=12)
plt.xticks(rotation=45, ha='right')  # Girar etiquetas para mayor legibilidad
plt.tight_layout()  # Ajustar layout
plt.show()

In [ ]:
# Parte 8: Tasa de conversión por educación y grupo de edad
tasa_conversion_educacion_grupo_edad = df.groupby(['education', 'grupo_edad'])['conversion'].mean().unstack() * 100
display(tasa_conversion_educacion_grupo_edad)

# Parte 9: Mapa de calor
import matplotlib.pyplot as plt
plt.figure(figsize=(14, 8))
plt.imshow(tasa_conversion_educacion_grupo_edad, cmap='YlGnBu', aspect='auto')
plt.title('Tasa de conversión por educación y grupo de edad', fontsize=16)
plt.xlabel('Grupo de edad', fontsize=12)
plt.ylabel('Nivel educativo', fontsize=12)
plt.xticks(ticks=range(len(tasa_conversion_educacion_grupo_edad.columns)), labels=tasa_conversion_educacion_grupo_edad.columns, rotation=45, ha='right')
plt.yticks(ticks=range(len(tasa_conversion_educacion_grupo_edad.index)), labels=tasa_conversion_educacion_grupo_edad.index)
plt.colorbar(label='Tasa de conversión (%)')
plt.tight_layout()
plt.show()